In [ ]:
import numpy as np
import pandas as pd
import glob
import tensorflow as tf
import cv2
from PIL import Image
from tensorflow.keras.applications.vgg16 import VGG16
import random
import os
import fnmatch

In [ ]:
batch_size = 64
auto = tf.data.AUTOTUNE

In [ ]:
monet_paths = glob.glob("../input/gan-getting-started/monet_jpg/*.jpg")
monet_paths.extend(glob.glob("../input/monet-paintings-jpg-berkeley/*.jpg"))

path = '../input/impressionist-classifier-data'
files_list = [os.path.join(dirpath, f)
    for dirpath, dirnames, files in os.walk(path)
    for f in fnmatch.filter(files, '*.jpg')]
monet_paths.extend(files_list)

path = '../input/van-gogh-paintings'
files_list = [os.path.join(dirpath, f)
    for dirpath, dirnames, files in os.walk(path)
    for f in fnmatch.filter(files, '*.jpg')]
monet_paths.extend(files_list)

path = '../input/impressionistlandscapespaintings/content/drive/MyDrive/impressionist_landscapes_resized_1024'
files_list = [os.path.join(dirpath, f)
    for dirpath, dirnames, files in os.walk(path)
    for f in fnmatch.filter(files, '*.jpg')]
monet_paths.extend(files_list)

photo_paths = glob.glob("../input/gan-getting-started/photo_jpg/*.jpg")

In [ ]:
def read_jpg(l):
    l = f"{l}"
    l = l[2:-1]
    image = cv2.imread(l)
    image = cv2.resize(image, (224,224))
    image = np.ndarray.astype(image, np.float32)
    
    return image

def read_jpg_tf(lt):
    image = tf.numpy_function(read_jpg, [lt], tf.float32)
    return image

In [ ]:
model = VGG16(weights='imagenet')
model.summary()

In [ ]:
def extract_feature(array):
    model = VGG16(weights='imagenet')
    model = tf.keras.Model(inputs=model.input, outputs=model.get_layer("block5_pool").output)
    model.trainable = False
    result = []

    pred = model.predict(array)
    result.append(pred)
    return np.squeeze(np.array(result), axis=0)

In [ ]:
photo_loader = tf.data.Dataset.list_files(photo_paths)

monet_loader = tf.data.Dataset.list_files(monet_paths)

In [ ]:
monet_loader = (monet_loader.map(read_jpg_tf)
                .shuffle(1024)
                .batch(batch_size, drop_remainder=True)
                .prefetch(auto))

photo_loader = (photo_loader.map(read_jpg_tf)
                .shuffle(1024)
                .batch(batch_size, drop_remainder=True)
                .prefetch(auto))

In [ ]:
photo_features = extract_feature(photo_loader)

In [ ]:
photo_features.shape

In [ ]:
photo_features_loader = tf.data.Dataset.from_tensor_slices(photo_features).shuffle(1024).batch(batch_size*2,  drop_remainder=True).prefetch(auto)

In [ ]:
generator = tf.keras.Sequential([
    tf.keras.Input(shape=[7, 7, 512]),
    tf.keras.layers.Conv2DTranspose(128, kernel_size=5, strides=2, padding="same", 
                                    activation="selu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2DTranspose(64, kernel_size=5, strides=2, padding="same", 
                                    activation="selu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2DTranspose(32, kernel_size=5, strides=2, padding="same", 
                                    activation="selu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2DTranspose(16, kernel_size=5, strides=2, padding="same", 
                                    activation="selu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2DTranspose(8, kernel_size=5, strides=2, padding="same", 
                                    activation="selu"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2DTranspose(3, kernel_size=5, strides=1, padding="same", 
                                    activation="tanh"),
])

In [ ]:
generator.summary()

In [ ]:
discriminator = tf.keras.Sequential([
    tf.keras.layers.Conv2D(8, kernel_size=5, strides=2, padding="same", 
                           activation=tf.keras.layers.LeakyReLU(0.2), input_shape=[224,224,3]),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(16, kernel_size=5, strides=2, padding="same", 
                           activation=tf.keras.layers.LeakyReLU(0.2)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(32, kernel_size=5, strides=2, padding="same", 
                           activation=tf.keras.layers.LeakyReLU(0.2)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(64, kernel_size=5, strides=2, padding="same", 
                           activation=tf.keras.layers.LeakyReLU(0.2)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(128, kernel_size=5, strides=2, padding="same", 
                           activation=tf.keras.layers.LeakyReLU(0.2)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(256, kernel_size=5, strides=2, padding="same", 
                           activation=tf.keras.layers.LeakyReLU(0.2)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
discriminator.summary()

In [ ]:
gan = tf.keras.Sequential([generator, discriminator])

In [ ]:
gan.build([None,7,7,512])

In [ ]:
def train_gan(gan, monet_loader, photo_features_loader, batch_size, epochs):
    gen, dis = gan.layers
    for epoch in range(epochs):
        for monet_batch, photo_batch in zip(monet_loader, photo_features_loader):
            #load feature data, reshape and concatenate
            generated_images = gen(photo_batch[:batch_size])
            generated_images = tf.cast(generated_images, tf.float32)
            monet_batch = tf.cast(monet_batch, tf.float32)
            mix_data = tf.concat([generated_images, monet_batch], axis=0)
            mix_label = tf.constant([[0.]]*batch_size + [[1.]]*batch_size)            
            #train discriminator
            dis.trainable = True
            dis.train_on_batch(mix_data, mix_label)
            #train generator
            generated_label = tf.constant([[1.]]*batch_size)
            dis.trainable = False
            gan.train_on_batch(photo_batch[batch_size:], generated_label)

In [ ]:
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable=False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [ ]:
train_gan(gan, monet_loader, photo_features_loader, batch_size, 3)

In [ ]:
tf.keras.models.save_model(gan, "GAN_PAST", save_format='h5')

In [ ]:
def produce(generator, photo_feature_loader):
    outputs = generator([photo_feature_loader])
    for n, output in enumerate(outputs):
        output.save(f"gan_{n}.jpg")

In [ ]:
inputs = next(iter(photo_features_loader))
outputs = generator([inputs])

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(outputs.numpy()[0])

In [ ]:
plt.imshow(outputs.numpy()[1])

In [ ]:
plt.imshow(photo[1001])

In [ ]:
outputs.numpy()[0]==outputs.numpy()[1]

In [ ]:
def produce_image(list):
    for l in list:
        image = generator(l)
        cv2.imwrite(".jpg", outputs